**Lab 8 : Latent Factor based Recommendation System**
- Instuctor : Kijung Shin
- Teaching Assistants : Hyunju Lee(main), Deukryeol Yoon, Shinhwan Kang 
- 본 실습에서는 추천 시스템에서 가장 흔히 쓰이는 surprise library를 사용하여 Latent Factor 기반 추천 시스템을 구현해본다.

In [1]:
!pip install surprise


     |████████████████████████████████| 11.8MB 5.3MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1615294 sha256=b1977fdc224176d35c773f5cf195424e66a931058643410fdfe28106bca73f45
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [2]:
import numpy as np 
import pandas as pd
from surprise import SVD
from surprise.model_selection import train_test_split
from surprise.dataset import DatasetAutoFolds
from surprise.model_selection import cross_validate
from surprise import Dataset, Reader
from surprise import accuracy


In [3]:
import os, sys 
from google.colab import drive 

### 해당 코드 실행 시 colab에서 실행중인 폴더의 /content/drive/My Drive가 구글 드라이브에 연결됨

drive.mount('/content/drive')

Mounted at /content/drive


**Dataset Loading**

In [4]:
#### 데이터셋 불러오기(MovieLens 10k) ####
df_ratings = pd.read_csv('drive/MyDrive/data/others/ratings.csv')

#### 평점 데이터셋 형태 확인#### 
# surprise library의 Reader 사용 시 반드시 사용자-아이템-평점 순으로 정보가 들어가 있어야 함

print("### Rating Dataset Format ###", end='\n\n')
print(df_ratings.head(), end='\n\n\n')
df_ratings.drop(['timestamp'], axis=1, inplace=True)
print("### Rating Dataset - Timestamp Removed ###", end='\n\n')
print(df_ratings)



### Rating Dataset Format ###

   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


### Rating Dataset - Timestamp Removed ###

        userId  movieId  rating
0            1        1     4.0
1            1        3     4.0
2            1        6     4.0
3            1       47     5.0
4            1       50     5.0
...        ...      ...     ...
100831     610   166534     4.0
100832     610   168248     5.0
100833     610   168250     5.0
100834     610   168252     5.0
100835     610   170875     3.0

[100836 rows x 3 columns]


In [5]:
df_movies = pd.read_csv('drive/MyDrive/data/others/movies.csv')

#### 영화 데이터셋 형태 확인 ####
print("### Movie Dataset Format ###", end = '\n\n')
print("Columns of Movie Dataset : ",df_movies.columns, end = '\n\n')
print(df_movies.head())

### Movie Dataset Format ###

Columns of Movie Dataset :  Index(['movieId', 'title', 'genres'], dtype='object')

   movieId  ...                                       genres
0        1  ...  Adventure|Animation|Children|Comedy|Fantasy
1        2  ...                   Adventure|Children|Fantasy
2        3  ...                               Comedy|Romance
3        4  ...                         Comedy|Drama|Romance
4        5  ...                                       Comedy

[5 rows x 3 columns]


In [6]:
#### Dataset의 User, Movie 수 확인 ####
n_users = df_ratings.userId.unique().shape[0]
n_items = df_ratings.movieId.unique().shape[0]
print("num users: {}, num items:{}".format(n_users, n_items))

num users: 610, num items:9724


In [7]:
### Add Your Own Data ### 

###################################### Example 1#################################################
# User 800 is a HUGE fan of Musical Movies
rows = []                               # row = [user_id, movie_id, rating]
user_id = 800
rows.append([user_id, 73, 5])        # movie    73: Miserables, Les (1995)
rows.append([user_id, 107780, 5])     # movie  107780: Cats(1998) 
rows.append([user_id, 588, 5])     # movie  588: Aladin(1992)
rows.append([user_id, 60397, 5])    # movie 69397: Mamma Mia!(2008)
rows.append([user_id, 99149, 5])    # movie 99149: Miserables, Les (2012)
rows.append([user_id, 138186, 1])    # movie 138186: Sorrow(2015)
rows.append([user_id, 1997, 1])    # movie 1997: Scream 2 (1991)

##################################################################################################

###################################### Example 2#################################################
# User 900 is a HUGE fan of Animation Movies
rows = []                               # row = [user_id, movie_id, rating]
user_id = 900
rows.append([user_id, 1022, 5])        # movie    1022: Cinderella(1950)
rows.append([user_id, 594, 5])     # movie  594: Snow White and the Seven Dwarfs(1937) 
rows.append([user_id, 106696, 5])     # movie  106696: Frozen(2013)
rows.append([user_id, 166461, 5])    # movie 166461: Moana(2016)
rows.append([user_id, 595, 5])    # movie 595: Beauty and the Beast (1991)
rows.append([user_id, 138168, 1])    # movie 138168: Sorrow(2015)
rows.append([user_id, 1997, 1])    # movie 1997: Scream 2 (1991)

##################################################################################################


########################### Add Your Own Ratings using 'movie.csv' data #########################
# my_rows = []
# my_id = 2021
# rows.append([user_id, ,])       # Fill your movie id and rating     
# rows.append([user_id, ,])       # 여러분이 평가할 영화의 id와 점수를 입력하세요.
# rows.append([user_id, ,])
# rows.append([user_id, ,])
# rows.append([user_id, ,])

##################################################################################################
for row in rows:
    df_ratings = df_ratings.append(pd.Series(row, index=df_ratings.columns), ignore_index=True)
print(df_ratings)

        userId  movieId  rating
0            1        1     4.0
1            1        3     4.0
2            1        6     4.0
3            1       47     5.0
4            1       50     5.0
...        ...      ...     ...
100838     900   106696     5.0
100839     900   166461     5.0
100840     900      595     5.0
100841     900   138168     1.0
100842     900     1997     1.0

[100843 rows x 3 columns]


In [8]:
#### Dataset의 User, Movie 수 확인 ####
n_users = df_ratings.userId.unique().shape[0]
n_items = df_ratings.movieId.unique().shape[0]
print("num users: {}, num items:{}".format(n_users, n_items))

num users: 611, num items:9725


In [10]:
#### Get Movid Name from Movie ID - 영화 ID로부터 영화 제목 얻기 ###

movie_set = set()     
ratings = np.zeros((n_users, n_items))
for (_, movie_id, _) in df_ratings.itertuples(index=False):
    movie_set.add(movie_id)

movie_id_to_name=dict()
movie_id_to_genre=dict()

for (movie_id, movie_name, movie_genre) in df_movies.itertuples(index=False):
    if movie_id not in movie_set:              # 어떤 영화가 rating data에 없는 경우 skip
        continue
    movie_id_to_name[movie_id] = movie_name 
    movie_id_to_genre[movie_id] = movie_genre

    


In [11]:
#### pandas dataframe을 surprise dataset 형태로 바꿔준 후, train set과 test set을 split 해준다 ####
reader = Reader(rating_scale=(0, 5)) # 평점 최고점 최저점
data = Dataset.load_from_df(df_ratings[['userId','movieId','rating']], reader=reader)

train, test = train_test_split(data, test_size=0.2, shuffle=True)

print(type(data))
print(type(train))

##################################################################################
## Grid Search를 위해 surprise.trainset 형태의 데이터를 surprise.dataset으로 변경해준다
iterator = train.all_ratings()
train_df = pd.DataFrame(columns=['userId', 'movieId', 'rating'])
i = 0
for (uid, iid, rating) in iterator:
    train_df.loc[i] = [train.to_raw_uid(int(uid)), train.to_raw_iid(iid), rating]
    i = i+1

train_data = Dataset.load_from_df(train_df, reader=reader)

print(type(train))
print(type(train_data))
##################################################################################


<class 'surprise.dataset.DatasetAutoFolds'>
<class 'surprise.trainset.Trainset'>
<class 'surprise.trainset.Trainset'>
<class 'surprise.dataset.DatasetAutoFolds'>


**모델 설정 및 학습**

In [20]:
### Hyperparameter Grid Search ### 

from surprise.model_selection import GridSearchCV
param_grid = {'n_factors': [10,15,20,30,50,100]} # 탐색의 공간 지정, 어느 차원을 embedding 차원으로 잡을 것인가?

####### Fill in Your Code ##########
grid = GridSearchCV(SVD, param_grid, measures = ['rmse'], cv=4) # 잠재 인수 모형 cv : cross-validation 시 4등분
#cross-validation(교차 검증) : 평가 데이터를 제외한 데이터를 n등분 한 뒤 1개를 제외한 데이터로 학습하고 제외한 1개로 hyperparameter를 정함, 이를 모든 경우의 수로 실행 (1,2,3 학습, 1,2,4 학습, 1,3,4, 학습, 2,3,4 g학습)
grid.fit(train_data)
#####################################


print(grid.best_score['rmse'])
print(grid.best_params['rmse'])

0.8771816744938526
{'n_factors': 10}


In [21]:
### Use the Hyperparameter with best performance ###

print(grid.best_params)
################ Fill in Your Code #################
algorithm = SVD(grid.best_params['rmse']['n_factors'])
algorithm.fit(train)


####################################################

{'rmse': {'n_factors': 10}}


**모델 예측**

In [22]:
##### algorithm prediction #####


prediction = algorithm.test(test)
for p in prediction[:5]:            # prediction 결과값 5개 미리보기
    print(p)                        # r_ui : 실제 rating 값, est: 예측된 rating 값
    

user: 372        item: 1274       r_ui = 1.00   est = 3.71   {'was_impossible': False}
user: 96         item: 1304       r_ui = 4.00   est = 4.14   {'was_impossible': False}
user: 124        item: 6377       r_ui = 5.00   est = 4.17   {'was_impossible': False}
user: 603        item: 4312       r_ui = 3.00   est = 3.53   {'was_impossible': False}
user: 384        item: 3527       r_ui = 2.00   est = 3.32   {'was_impossible': False}


In [23]:
#### 특정 user, 특정 item에 대한 prediction 값 ###
uid = 800
iid = 8368
prediction_user_item = algorithm.predict(uid, iid)
print(prediction_user_item)     

user: 800        item: 8368       r_ui = None   est = 4.05   {'was_impossible': False}


In [19]:
##############################################################
##### 해당 user가 아직 보지 않은 영화를 return해주는 함수#####
##############################################################
def get_unseen_movies(data, user_id):

    watched_movies = set()
    total_movies = set()
    ########### Fill in Your Code #################


    ##################################################
    return unseen_movies
    # return total_movies

In [24]:
################################################################################
############# 특정 user에게 top k개의 영상을 추천해주는 함수 ###################
################################################################################
def recommend(train, algorithm, user_id, top_k=10):
    ################ Fill in Your Code ########################################


    ###########################################################################
    for _, movie, _, pred, _ in prediction[:top_k]:
        print("movid id: {}, movie genre: {},predicted rating: {}".format(movie_id_to_name[movie], movie_id_to_genre[movie], pred))


In [25]:
#########################################
####### 800번 유저의 추천 결과 ##########
#########################################

recommend(train, algorithm, user_id=800, top_k=20)


movid id: Akira (1988), movie genre: Action|Adventure|Animation|Sci-Fi,predicted rating: 3.7146585424571033
movid id: Butch Cassidy and the Sundance Kid (1969), movie genre: Action|Western,predicted rating: 4.139810001683979
movid id: Finding Nemo (2003), movie genre: Adventure|Animation|Children|Comedy,predicted rating: 4.1660039711449315
movid id: Himalaya (Himalaya - l'enfance d'un chef) (1999), movie genre: Adventure|Drama,predicted rating: 3.5332903240563533
movid id: Predator (1987), movie genre: Action|Sci-Fi|Thriller,predicted rating: 3.324864886541601
movid id: Billy Madison (1995), movie genre: Comedy,predicted rating: 3.2060433016846126
movid id: Rules of the Game, The (La règle du jeu) (1939), movie genre: Comedy|Drama,predicted rating: 3.520060082088648
movid id: Stakeout (1987), movie genre: Comedy|Crime|Romance|Thriller,predicted rating: 3.0709777973429357
movid id: Awful Truth, The (1937), movie genre: Comedy|Romance,predicted rating: 3.323286845514057
movid id: Big Leb

In [26]:
#########################################
####### 900번 유저의 추천 결과 ##########
#########################################

recommend(train, algorithm, user_id=900, top_k=20)


movid id: Akira (1988), movie genre: Action|Adventure|Animation|Sci-Fi,predicted rating: 3.7146585424571033
movid id: Butch Cassidy and the Sundance Kid (1969), movie genre: Action|Western,predicted rating: 4.139810001683979
movid id: Finding Nemo (2003), movie genre: Adventure|Animation|Children|Comedy,predicted rating: 4.1660039711449315
movid id: Himalaya (Himalaya - l'enfance d'un chef) (1999), movie genre: Adventure|Drama,predicted rating: 3.5332903240563533
movid id: Predator (1987), movie genre: Action|Sci-Fi|Thriller,predicted rating: 3.324864886541601
movid id: Billy Madison (1995), movie genre: Comedy,predicted rating: 3.2060433016846126
movid id: Rules of the Game, The (La règle du jeu) (1939), movie genre: Comedy|Drama,predicted rating: 3.520060082088648
movid id: Stakeout (1987), movie genre: Comedy|Crime|Romance|Thriller,predicted rating: 3.0709777973429357
movid id: Awful Truth, The (1937), movie genre: Comedy|Romance,predicted rating: 3.323286845514057
movid id: Big Leb